In [ ]:
import glob, os, gc
import os.path
import csv
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from time import time
from subprocess import (call, Popen, PIPE)
from itertools import product
from sklearn import (manifold, datasets, decomposition, ensemble, discriminant_analysis, random_projection)
from sklearn.decomposition import (PCA, RandomizedPCA)
from sklearn.datasets import fetch_mldata
from sklearn.utils import shuffle
from IPython.display import Image
from PIL import Image
from IPython.display import Image as IPImage
import shutil
import pybedtools
import pysam
import re
import xml.etree.ElementTree as ET
import time



##Path to Data
basepath = "/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/SLAV_Data/" 
narrowpeak = "-ready_peaks.narrowPeak"
peaks_merged = "_peaksMerged.txt"
peaks_merged_bed = "_peaksMerged.bed"
peaks_correct_bed = "_peaksCorrect.bed"
peakregions_sml = ".peakregions_sml"
peakregions_lrg = ".peakregions_lrg"
peaks_correct_data = "_peaksCorrect.data"
peaks_L1HS_bedgraph = "_peaks_L1HS_mapped.bedgraph"
loci_sml = ".loci_sml"
loci_lrg = ".loci_lrg"
overlap = "_overlap_"
overlap_sml = "_overlap_sml_"
overlap_lrg = "_overlap_lrg_"
L1HS_sam = "-L1HS_mapped.sam"
bam = "-ready.bam"
igv = "-igv.xml"
bed = ".bed"
## rmask Paths 
L1HS = "/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/rmask_L1HS_Final.bed"
L1PA2345 = "/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/rmask_L1PA2345_Final.bed"
L1_Other = "/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/rmask_L1_Other_Final.bed"
##IGV Template
IGV = "/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/igv-template4.xml"

Bulk_1571_Cerebellum = "1571_cereb_BT_40_L3"
Bulk_1571_Hippocampus = "1571_hippo_BT_41_L3"
SC_1571_Hippo = ["1571_hippo_SC_43_L3","1571_hippo_SC_45_L3","1571_hippo_SC_46_L3","1571_hippo_SC_47_L3","1571_hippo_SC_48_L3","1571_hippo_SC_50_L3","1571_hippo_SC_51_L3","1571_hippo_SC_52_L3","1571_hippo_SC_53_L3","1571_hippo_SC_55_L3","1571_hippo_SC_56_L3","1571_hippo_SC_57_L3","1571_hippo_SC_58_L3","1571_hippo_SC_59_L3","1571_hippo_SC_61_L3","1571_hippo_SC_62_L3","1571_hippo_SC_63_L3","1571_hippo_SC_64_L3"]


Data_Sets = []
Data_Sets.append([SC_1571_Hippo,Bulk_1571_Hippocampus,Bulk_1571_Cerebellum])

In [ ]:
for dset in Data_Sets:
    for cell in dset[0]:
        print cell
        tree = ET.parse(IGV)
        root = tree.getroot()
        root[0][0].set('path', os.path.join(basepath,  cell, cell + peaks_L1HS_bedgraph)) #L1HS bedgraph
        root[0][1].set('path', os.path.join(basepath,  cell, cell + bam)) #SC Path
        root[0][3].set('path', os.path.join(basepath,  dset[1], dset[1] + bam)) #Bulk Brain Path
        root[0][6].set('path', os.path.join(basepath,  dset[2], dset[2] + bam)) #Bulk Fib Path
        root[1][0].set('id', os.path.join(basepath,  cell, cell + peaks_L1HS_bedgraph)) #L1HS bedgraph
        root[2][0].set('id', os.path.join(basepath,  cell, cell + bam)) #SC Path
        root[3][0].set('id', os.path.join(basepath,  dset[1], dset[1] + bam)) #Bulk Brain path
        root[4][0].set('id', os.path.join(basepath,  dset[2], dset[2] + bam)) #Bulk Fib Path
        tree.write(os.path.join(basepath,  cell, cell + igv))



        myinput = open(os.path.join(basepath,  cell, cell + peaks_merged_bed))
        myoutput = open(os.path.join(basepath,  cell, cell + peaks_correct_bed), 'w')
        proc3 = Popen(['grep', '-E', '^(1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19|20|21|22|X|Y)'], stdin=myinput, stdout=myoutput)
        proc3.wait()    

        myinput = os.path.join(basepath,  cell, cell + peaks_correct_bed)
        myoutput1 = os.path.join(basepath,  cell, cell + peakregions_sml)
        myoutput2 = os.path.join(basepath,  cell, cell + peakregions_lrg)
        with open(myoutput1, 'w') as outfile:
            with open(myinput, 'r') as infile:
                data = infile.readlines()
                for string in data:
                    line = string.split('\t')
                    pos1 = int(line[1])
                    pos2 = int(line[2])                  
                    center = int((pos1 + pos2)/2)
                    pad = 1000
                    start = center - pad
                    end = center + pad
                    row = [line[0], str(start), str(end)]
                    outfile.write('\t'.join(row) + '\n')
        outfile.close()
        infile.close()   
        with open(myoutput2, 'w') as outfile:
            with open(myinput, 'r') as infile:
                data = infile.readlines()
                for string in data:
                    line = string.split('\t')
                    pos1 = int(line[1])
                    pos2 = int(line[2])                  
                    center = int((pos1 + pos2)/2)
                    pad = 10000
                    newstart = center - pad
                    newend = center + pad
                    row = [line[0], str(newstart), str(newend)]
                    outfile.write('\t'.join(row) + '\n')
        outfile.close()
        infile.close()
 
        #make and define L1HS sub sam file here and L1HS read names list
        
               
        sc_file = pysam.AlignmentFile(os.path.join(basepath,  cell, cell + bam), "rb")
        bb_file = pysam.AlignmentFile(os.path.join(basepath,  dset[1], dset[1] + bam), "rb")
        bf_file = pysam.AlignmentFile(os.path.join(basepath,  dset[2], dset[2] + bam), "rb")
        

        myinput = os.path.join(basepath,  cell, cell + peaks_correct_bed)
        myoutput = os.path.join(basepath,  cell, cell + peaks_correct_data)
        
        with open(myoutput, 'w') as outfile:
            with open(myinput, 'r') as infile:
                data = infile.readlines()
                for region in data:
                    sc_iter = sc_file.fetch(region.split('\t')[0], int(region.split('\t')[1]), int(region.split('\t')[2]))
                    bb_iter = bb_file.fetch(region.split('\t')[0], int(region.split('\t')[1]), int(region.split('\t')[2]))
                    bf_iter = bf_file.fetch(region.split('\t')[0], int(region.split('\t')[1]), int(region.split('\t')[2]))
                    sc_i = 0
                    bb_i = 0
                    bf_i = 0
                    for x in sc_iter: sc_i+=1
                    for y in bb_iter: bb_i+=1
                    for z in bf_iter: bf_i+=1
                    sc_count = 1 if sc_i > 0 else 0
                    bb_count = 1 if bb_i > 0 else 0 
                    bf_count = 1 if bf_i > 0 else 0 
                    row = [str(region.strip().split('\t')[0]), str(region.strip().split('\t')[1]), str(region.strip().split('\t')[2])]  
                    outfile.write('\t'.join(row) +'\t'+str(sc_count)+str(bb_count)+str(bf_count)+'\n')
        
        myinput = os.path.join(basepath,  cell, cell + peaks_correct_bed)
        myoutput = open(os.path.join(basepath, cell, cell + L1HS_sam), 'w')
        myoutput2 = os.path.join(basepath,  cell, cell + peaks_L1HS_bedgraph)
        p1 = Popen(['samtools', 'view', '-h', '-L', L1HS, os.path.join(basepath,  cell, cell + bam)], stdout=myoutput)
        p1.wait()
        L1HS_file = pysam.AlignmentFile(os.path.join(basepath, cell, cell + L1HS_sam), 'r')
        L1HS_read_names =[]
        for read in L1HS_file.fetch(): 
            read_name = str(read).split('\t')[0]
            L1HS_read_names.append(read_name)
        with open(myoutput2, 'w') as outfile:
            with open(myinput, 'r') as infile:
                data = infile.readlines()
                for region in data:
                    read_names = []
                    for read in sc_file.fetch(region.split('\t')[0], int(region.split('\t')[1]), int(region.split('\t')[2])):
                        read_name = str(read).split('\t')[0]
                        read_names.append(read_name)
                    L1HS_overlap = list(set(read_names) & set(L1HS_read_names))
                    percent = (len(L1HS_overlap) / len(read_names))*100
                    row = [str(region.strip().split('\t')[0]), str(region.strip().split('\t')[1]), str(region.strip().split('\t')[2]), str(percent)]  
                    outfile.write('\t'.join(row)+'\n')

        filelist =[os.path.join(basepath,  cell, "peaks.bed"),os.path.join(basepath,  dset[1], "peaks.bed"),os.path.join(basepath,  dset[2], "peaks.bed"),L1HS,L1PA2345,L1_Other]
        a = pybedtools.BedTool(os.path.join(basepath, cell, cell + peaks_correct_bed)) ##mergedpeaks2 overlap with loci window or with peak location???
        count = 0
        for fname in filelist:
            b = pybedtools.BedTool(fname)
            a_and_b = a.intersect(b, c=True)
            myoutput = os.path.join(basepath,  cell, cell + overlap + str(count))
            count +=1
            a_and_b.saveas(myoutput)
            myinput = open(myoutput)
            newoutput = open(myoutput+"_binary", 'w')
            #overlap_append
            awk_cmd = r"""BEGIN { OFS = "\t"; }; { if ($7 ~ "^[1-9]*$") $7 = "1"; else $7 = $7; }; 1"""
            proc = Popen(['awk', awk_cmd], stdin=myinput, stdout=newoutput)  
            proc.wait()
            newoutput.flush()

        a_sml = pybedtools.BedTool(os.path.join(basepath,  cell, cell + peakregions_sml)) ##mergedpeaks2 overlap with loci window or with peak location???
        count = 0
        for fname in filelist:
            b = pybedtools.BedTool(fname)
            a_and_b = a_sml.intersect(b, c=True)
            myoutput = os.path.join(basepath,  cell, cell + overlap_sml + str(count))
            count +=1
            a_and_b.saveas(myoutput)
            myinput = open(myoutput)
            newoutput = open(myoutput+"_binary", 'w')
            #overlap_append
            awk_cmd = r"""BEGIN { OFS = "\t"; }; { if ($4 >= 2) $4 = "2"; else $4 = $4; }; 1"""
            proc = Popen(['awk', awk_cmd], stdin=myinput, stdout=newoutput)  
            proc.wait()
            newoutput.flush()

        a_lrg = pybedtools.BedTool(os.path.join(basepath,  cell, cell + peakregions_lrg)) ##mergedpeaks2 overlap with loci window or with peak location???
        count = 0
        for fname in filelist:
            b = pybedtools.BedTool(fname)
            a_and_b = a_lrg.intersect(b, c=True)
            myoutput = os.path.join(basepath,  cell, cell + overlap_lrg + str(count))
            count +=1
            a_and_b.saveas(myoutput)
            myinput = open(myoutput)
            newoutput = open(myoutput+"_binary", 'w')
            #overlap_append
            awk_cmd = r"""BEGIN { OFS = "\t"; }; { if ($4 >= 2) $4 = "2"; else $4 = $4; }; 1"""
            proc = Popen(['awk', awk_cmd], stdin=myinput, stdout=newoutput)  
            proc.wait()
            newoutput.flush()        

        myinput_sml = os.path.join(basepath,  cell, cell + peakregions_sml)
        myoutput_sml = os.path.join(basepath,  cell, cell + loci_sml)
        with open(myoutput_sml, 'w') as outfile:
            with open(myinput_sml, 'r') as infile:
                data = infile.readlines()
                for region in data:
                    row = [str(region.strip().split('\t')[0]),":",str(region.strip().split('\t')[1]),"-",str(region.strip().split('\t')[2])]  
                    outfile.write("".join(row)+'\n')

        myinput_lrg = os.path.join(basepath,  cell, cell + peakregions_lrg)
        myoutput_lrg = os.path.join(basepath,  cell, cell + loci_lrg)
        with open(myoutput_lrg, 'w') as outfile:
            with open(myinput_lrg, 'r') as infile:
                data = infile.readlines()
                for region in data:
                    row = [str(region.strip().split('\t')[0]),":",str(region.strip().split('\t')[1]),"-",str(region.strip().split('\t')[2])]  
                    outfile.write("".join(row)+'\n')    

        Popen(['split', '-l', '100', '-d', os.path.join(basepath,  cell, cell + loci_sml), os.path.join(basepath,  cell, cell + ".split_loci_sml_")]).wait()
        Popen(['split', '-l', '100', '-d', os.path.join(basepath,  cell, cell + loci_lrg), os.path.join(basepath,  cell, cell + ".split_loci_lrg_")]).wait()

In [ ]:
for dset in Data_Sets:    
    for cell in dset[0]:
        print cell
        os.chdir(os.path.join(basepath, cell))    
        locifile = os.path.join(basepath, cell, cell + loci_sml)
        worklist = glob.glob("*.split_loci_sml_*")
        batchsize = 10
        print len(worklist)
        for i in xrange(0, len(worklist), batchsize):
            batch = worklist[i:i+batchsize]
            print i
            index = 1
            procs = []
            for file in batch:
                print file
                with open(os.path.join(basepath, cell, file)) as f0:
                    first = f0.readline()# Read the first line.
                    for last in f0: pass
                    firstpic = cell+"_sml"+"*"+first.strip().split(':')[0]+"_"+first.strip().split(':')[1].split('-')[0]+"_"+first.strip().split(':')[1].split('-')[1]+".png"
                    lastpic = cell+"_sml"+"*"+last.strip().split(':')[0]+"_"+last.strip().split(':')[1].split('-')[0]+"_"+last.strip().split(':')[1].split('-')[1]+".png"
                    if not (glob.glob(os.path.join(basepath, cell, firstpic)) or glob.glob(os.path.join(basepath, cell, lastpic))): 
                        p = Popen(['igv_plotter', '-o', cell+"_sml_", '-L', file, '-v', '--max-panel-height', '1000', '--igv-jar-path', '/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/IGV_2.4-rc6/igv.jar', '-m', '6G', '-g', 'hg19', os.path.join(basepath, cell, cell + igv)])
                        procs.append(p)
            for pp in procs:
                pp.wait()
                #wait_timeout(pp,300)

In [ ]:
for cell in SingleCell_MDA:
    print os.path.join(basepath, diry, cell)
    os.chdir(os.path.join(basepath, diry, cell))    
    locifile = os.path.join(basepath, diry, cell, cell + loci_lrg)
    worklist = glob.glob("*.split_loci_lrg_*")
    batchsize = 10
    print len(worklist)
    for i in xrange(0, len(worklist), batchsize):
        batch = worklist[i:i+batchsize]
        print i
        index = 1
        procs = []
        for file in batch:
            print file
            with open(os.path.join(basepath, diry, cell, file)) as f0:
                first = f0.readline()# Read the first line.
                for last in f0: pass
                firstpic = cell+"_lrg"+"*"+first.strip().split(':')[0]+"_"+first.strip().split(':')[1].split('-')[0]+"_"+first.strip().split(':')[1].split('-')[1]+".png"
                lastpic = cell+"_lrg"+"*"+last.strip().split(':')[0]+"_"+last.strip().split(':')[1].split('-')[0]+"_"+last.strip().split(':')[1].split('-')[1]+".png"
                if not (glob.glob(os.path.join(basepath, diry, cell, firstpic)) or glob.glob(os.path.join(basepath, diry, cell, lastpic))): 
                    p = Popen(['igv_plotter', '-o', cell+"_lrg_", '-L', file, '-v', '--max-panel-height', '1000', '--igv-jar-path', '/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/Common_Experiment/IGV_2.4-rc6/igv.jar', '-m', '6G', '-g', 'hg19', os.path.join(basepath, diry, cell, cell + igv)])
                    procs.append(p)
        for pp in procs:
            pp.wait()
            #wait_timeout(pp,300)

In [ ]:
for cell in SingleCell_MDA:
    print cell
    os.chdir(os.path.join(basepath, diry, cell))
    for file in glob.glob("*s*__*.png"):
        newfile = re.sub("_s\d+__", "-", file)
        shutil.move(file, newfile)     
    for file in glob.glob("*.png"):
        img = Image.open(file)
        width = img.size[0]
        height = img.size[1]
        img2 = img.crop((70,130,width,height)).resize((200,500))
        path = os.path.splitext(file)[0]
        basename = os.path.basename(path)
        outfile1 = basename + "_crp.png"
        img2.save(outfile1)
        #os.remove(file)
    
    mergedpeak_data = os.path.join(basepath, diry, cell, cell + peaks_correct_data)
    regions_sml = os.path.join(basepath, diry, cell, cell + peakregions_sml)
    regions_lrg = os.path.join(basepath, diry, cell, cell + peakregions_lrg)
      
    count=1
    with open(mergedpeak_data) as r0,open(regions_sml) as r_sml,open(regions_lrg) as r_lrg:
        Files= {}
        for rr0,rr_sml,rr_lrg in zip(r0,r_sml,r_lrg):
            line = rr0.strip().split('\t')[0]+"\t"+rr0.strip().split('\t')[1]+"\t"+rr0.strip().split('\t')[2]+"\t"+cell+"_sml-"+rr_sml.strip().split('\t')[0]+"_"+rr_sml.strip().split('\t')[1]+"_"+rr_sml.strip().split('\t')[2]+"_crp.png"+"\t"+cell+"_lrg-"+rr_lrg.strip().split('\t')[0]+"_"+rr_lrg.strip().split('\t')[1]+"_"+rr_lrg.strip().split('\t')[2]+"_crp.png"+"\t"+rr0.strip().split('\t')[3]
            Files[str(count)] = line
            count+=1
    
    a = os.path.join(basepath, diry, cell, cell+"_overlap_0_binary")
    b = os.path.join(basepath, diry, cell, cell+"_overlap_1_binary")
    c = os.path.join(basepath, diry, cell, cell+"_overlap_2_binary")
    d = os.path.join(basepath, diry, cell, cell+"_overlap_3_binary")
    e = os.path.join(basepath, diry, cell, cell+"_overlap_4_binary")
    f = os.path.join(basepath, diry, cell, cell+"_overlap_5_binary")
    count=1
    with open(a) as f1,open(b) as f2,open(c) as f3,open(d) as f4,open(e) as f5,open(f) as f6:
        Peaks = {}
        for aa,bb,cc,dd,ee,ff in zip(f1,f2,f3,f4,f5,f6):
            line = aa.strip().split('\t')[6]+bb.strip().split('\t')[6]+cc.strip().split('\t')[6]+"\t"+dd.strip().split('\t')[6]+ee.strip().split('\t')[6]+ff.strip().split('\t')[6]
            Peaks[str(count)] = line
            count+=1
    
    a_sml = os.path.join(basepath, diry, cell, cell+"_overlap_sml_0_binary")
    b_sml = os.path.join(basepath, diry, cell, cell+"_overlap_sml_1_binary")
    c_sml = os.path.join(basepath, diry, cell, cell+"_overlap_sml_2_binary")
    d_sml = os.path.join(basepath, diry, cell, cell+"_overlap_sml_3_binary")
    e_sml = os.path.join(basepath, diry, cell, cell+"_overlap_sml_4_binary")
    f_sml = os.path.join(basepath, diry, cell, cell+"_overlap_sml_5_binary")
    count=1
    with open(a_sml) as f1,open(b_sml) as f2,open(c_sml) as f3,open(d_sml) as f4,open(e_sml) as f5,open(f_sml) as f6:
        Small = {}
        for aa,bb,cc,dd,ee,ff in zip(f1,f2,f3,f4,f5,f6):
            line = aa.strip().split('\t')[3]+bb.strip().split('\t')[3]+cc.strip().split('\t')[3]+"\t"+dd.strip().split('\t')[3]+ee.strip().split('\t')[3]+ff.strip().split('\t')[3]
            Small[str(count)] = line
            count+=1
    
    a_lrg = os.path.join(basepath, diry, cell, cell+"_overlap_lrg_0_binary")
    b_lrg = os.path.join(basepath, diry, cell, cell+"_overlap_lrg_1_binary")
    c_lrg = os.path.join(basepath, diry, cell, cell+"_overlap_lrg_2_binary")
    d_lrg = os.path.join(basepath, diry, cell, cell+"_overlap_lrg_3_binary")
    e_lrg = os.path.join(basepath, diry, cell, cell+"_overlap_lrg_4_binary")
    f_lrg = os.path.join(basepath, diry, cell, cell+"_overlap_lrg_5_binary")
    count=1
    with open(a_lrg) as f1,open(b_lrg) as f2,open(c_lrg) as f3,open(d_lrg) as f4,open(e_lrg) as f5,open(f_lrg) as f6:
        Large = {}
        for aa,bb,cc,dd,ee,ff in zip(f1,f2,f3,f4,f5,f6):
            line = aa.strip().split('\t')[3]+bb.strip().split('\t')[3]+cc.strip().split('\t')[3]+"\t"+dd.strip().split('\t')[3]+ee.strip().split('\t')[3]+ff.strip().split('\t')[3]
            Large[str(count)] = line
            count+=1
    
    with open(os.path.join(basepath, diry, cell, cell+"_Input_metadata.txt"),"w") as f8:
        for key in Files:
            encoding = Files[key]+":"+Peaks[key].strip().split('\t')[0]+":"+Large[key].strip().split('\t')[0]+":"+Peaks[key].strip().split('\t')[1]+"\n"
        #for (Fi, Fj),(Pi,Pj),(Si,Sj),(Li,Lj) in zip(Files.items(),Peaks.items(),Small.items(),Large.items()):
            #encoding = Fj+":"+Pj.strip().split('\t')[0]+":"+Sj.strip().split('\t')[0]+":"+Lj.strip().split('\t')[0]+":"+Pj.strip().split('\t')[1]+":"+Sj.strip().split('\t')[1]+":"+Lj.strip().split('\t')[1]+"\n"
            f8.write(encoding)
            
#    with open(os.path.join(basepath, diry, cell, cell+"_Input_metadata.txt")) as f:
#        for line in csv.reader(f, delimiter="\t"):
#            if os.path.isfile(line[3]):
#                filename = line[3]
#                dst = line[5]
#                if not os.path.exists(dst):
#                    os.makedirs(os.path.join(basepath, diry, cell, dst))
#                shutil.move(filename, os.path.join(basepath, diry, cell, dst))
                       
#    with open(os.path.join(basepath, diry, cell, cell+"_Input_metadata.txt")) as f:
#        for line in csv.reader(f, delimiter="\t"):
#            if os.path.isfile(line[4]):
#                filename = line[4]
#                dst = line[5]
#                if not os.path.exists(dst):
#                    os.makedirs(os.path.join(basepath, diry, cell, dst))
#                shutil.move(filename, os.path.join(basepath, diry, cell, dst))                
        